In [59]:
import pandas as pd
from bs4 import BeautifulSoup
import time
import datetime

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [60]:
df = pd.read_csv('../data/raw/coinmarketcap_cryptocurrency_listings_latest.csv')
df.head()

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote
0,1,Bitcoin,BTC,bitcoin,10943,2010-07-13T00:00:00.000Z,"['mineable', 'pow', 'sha-256', 'store-of-value...",21000000.0,1.966502e+07,1.966502e+07,False,NaN,1,NaN,NaN,NaN,2024-03-26T18:25:00.000Z,"{'USD': {'price': 70215.2092211454, 'volume_24..."
1,1027,Ethereum,ETH,ethereum,8652,2015-08-07T00:00:00.000Z,"['pos', 'smart-contracts', 'ethereum-ecosystem...",NaN,1.200740e+08,1.200740e+08,True,NaN,2,NaN,NaN,NaN,2024-03-26T18:24:00.000Z,"{'USD': {'price': 3590.2311684461333, 'volume_..."
2,825,Tether USDt,USDT,tether,80280,2015-02-25T00:00:00.000Z,"['payments', 'stablecoin', 'asset-backed-stabl...",NaN,1.042337e+11,1.077926e+11,True,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",3,NaN,NaN,NaN,2024-03-26T18:24:00.000Z,"{'USD': {'price': 1.000137662205108, 'volume_2..."
3,1839,BNB,BNB,bnb,2124,2017-07-25T00:00:00.000Z,"['marketplace', 'centralized-exchange', 'payme...",NaN,1.495367e+08,1.495367e+08,False,NaN,4,NaN,NaN,NaN,2024-03-26T18:24:00.000Z,"{'USD': {'price': 579.0937507284682, 'volume_2..."
4,5426,Solana,SOL,solana,646,2020-04-10T00:00:00.000Z,"['pos', 'platform', 'solana-ecosystem', 'cms-h...",NaN,4.441814e+08,5.725844e+08,True,NaN,5,NaN,NaN,NaN,2024-03-26T18:24:00.000Z,"{'USD': {'price': 189.31256529881077, 'volume_..."


# Get historical data from 2020-2024 for each currency using url

In [61]:
base_url = 'https://coinmarketcap.com'

In [62]:
start_date = 20200101
end_date = 202403026

In [63]:
historical_url = f'{base_url}/currencies/bitcoin/historical-data/?start={start_date}&end={end_date}'
historical_url

'https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20200101&end=202403026'

In [69]:
driver = webdriver.Chrome()
driver.get(historical_url)

In [71]:
# wait for the history table to be generated
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.history table')))


<selenium.webdriver.remote.webelement.WebElement (session="6d546be487d7993c6f77509a779d6160", element="f.EFE5F0684A142AC61E70F790ED2F8156.d.8F5BF0EC361BC391F5C056B50B37C9FC.e.22")>

In [72]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [74]:
history_div = soup.find('div', {'class': 'history'})
history_table = history_div.find('table')
history_table

<table class="sc-14cb040a-3 eGIvUX cmc-table"><colgroup><col style="width: 315px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 315px; min-width: auto;"/><col style="width: 150px; min-width: auto;"/></colgroup><thead><tr><th class="stickyTop" style="text-align: start; top: 1px;">Date</th><th class="stickyTop" style="text-align: start; top: 1px;">Open*</th><th class="stickyTop" style="text-align: start; top: 1px;">High</th><th class="stickyTop" style="text-align: end; top: 1px;">Low</th><th class="stickyTop" style="text-align: end; top: 1px;">Close**</th><th class="stickyTop" style="text-align: end; top: 1px;">Volume</th><th class="stickyTop" style="text-align: end; top: 1px;">Market Cap</th></tr></thead><tbody><tr><td style="text-align: start;">Mar 26, 2024</td><td style="text-align: start;">$69,931.33</td><td style=

In [88]:
history_table.findAll('tr')[-1].find('td').text

'Jan 28, 2024'

In [83]:
history_div.findAll('table tr')[-1].find_elements_by_tag_name('td')[0].text

IndexError: list index out of range

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
history_div = soup.find('div', {'class': 'history'})
history_table = history_div.find('table')

# Get the last date in the table
last_date_string = history_div.find(' table tr')[-1].find_elements_by_tag_name('td')[0].text
last_date = datetime.strptime(last_date_string, '%b %d, %Y')
print("Last date in the table: ", last_date)
print("start_date: ", start_date)

if last_date <= start_date:
    print('End reached')
print('End not reached')

In [66]:
def check_if_limit_reached(driver, start_date):
    
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        history_div = soup.find('div', {'class': 'history'})
        history_table = history_div.find('table')

        # Get the last date in the table
        last_date_string = history_div.find(' table tr')[-1].find_elements_by_tag_name('td')[0].text
        last_date = datetime.strptime(last_date_string, '%b %d, %Y')
        print("Last date in the table: ", last_date)
        print("start_date: ", start_date)
        
        if last_date <= start_date:
            return True
        return False
    
    except Exception as e:
        print('Exception check_if_limit_reached: ', e)
        return False

In [67]:
# while True:
#     try:
#         # wait for load_more_button to be clickable
#         load_more_button = wait.until(EC.presence_of_element_located((By.XPATH, "//button[text()='Load More']")))
#         driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
#         driver.execute_script("arguments[0].click();", load_more_button)
#         time.sleep(3)
        
#         if check_if_limit_reached(driver, start_date):
#             break
        
#     except Exception as e:
#         break

Exception check_if_limit_reached:  'NoneType' object is not subscriptable
Exception check_if_limit_reached:  'NoneType' object is not subscriptable
Exception check_if_limit_reached:  'NoneType' object is not subscriptable
Exception check_if_limit_reached:  'NoneType' object is not subscriptable
Exception check_if_limit_reached:  'NoneType' object is not subscriptable
Exception check_if_limit_reached:  'NoneType' object is not subscriptable
Exception check_if_limit_reached:  'NoneType' object is not subscriptable
Exception check_if_limit_reached:  'NoneType' object is not subscriptable
Exception check_if_limit_reached:  'NoneType' object is not subscriptable
Exception check_if_limit_reached:  'NoneType' object is not subscriptable
Exception check_if_limit_reached:  'NoneType' object is not subscriptable
Exception check_if_limit_reached:  'NoneType' object is not subscriptable
Exception check_if_limit_reached:  'NoneType' object is not subscriptable
Exception check_if_limit_reached:  'No

In [68]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.59)
Stacktrace:
0   chromedriver                        0x00000001025ac41c chromedriver + 4326428
1   chromedriver                        0x00000001025a48e4 chromedriver + 4294884
2   chromedriver                        0x00000001021d0088 chromedriver + 278664
3   chromedriver                        0x00000001021ab270 chromedriver + 127600
4   chromedriver                        0x00000001022379ac chromedriver + 702892
5   chromedriver                        0x000000010224ac0c chromedriver + 781324
6   chromedriver                        0x00000001022074e4 chromedriver + 505060
7   chromedriver                        0x0000000102207f5c chromedriver + 507740
8   chromedriver                        0x000000010256f9b8 chromedriver + 4078008
9   chromedriver                        0x0000000102574770 chromedriver + 4097904
10  chromedriver   

In [ ]:
history_div = soup.find('div', {'class': 'history'})
history_table = history_div.find('table')
print(history_table)

<table class="sc-14cb040a-3 eGIvUX cmc-table"><colgroup><col style="width: 315px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 315px; min-width: auto;"/><col style="width: 150px; min-width: auto;"/></colgroup><thead><tr><th class="stickyTop" style="text-align: start; top: 1px;">Date</th><th class="stickyTop" style="text-align: start; top: 1px;">Open*</th><th class="stickyTop" style="text-align: start; top: 1px;">High</th><th class="stickyTop" style="text-align: end; top: 1px;">Low</th><th class="stickyTop" style="text-align: end; top: 1px;">Close**</th><th class="stickyTop" style="text-align: end; top: 1px;">Volume</th><th class="stickyTop" style="text-align: end; top: 1px;">Market Cap</th></tr></thead><tbody><tr><td style="text-align: start;">Mar 26, 2024</td><td style="text-align: start;">$69,931.33</td><td style=

In [ ]:
history_df = pd.read_html(str(history_table))[0]
history_df.head()

/var/folders/qn/zrg5ms9n4mj424yzj3h57fh80000gn/T/ipykernel_21584/102780542.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  history_df = pd.read_html(str(history_table))[0]


,Date,Open*,High,Low,Close**,Volume,Market Cap
0,"Mar 26, 2024","$69,931.33","$71,535.74","$69,335.61","$69,987.84","$36,010,437,368","$1,376,403,313,500"
1,"Mar 25, 2024","$67,234.10","$71,162.59","$66,414.83","$69,958.81","$42,700,139,523","$1,375,150,091,237"
2,"Mar 24, 2024","$64,070.75","$67,622.76","$63,825.85","$67,234.17","$27,206,630,673","$1,322,058,484,489"
3,"Mar 23, 2024","$63,802.72","$65,976.40","$63,038.49","$64,062.20","$24,738,964,812","$1,259,788,729,091"
4,"Mar 22, 2024","$65,489.93","$66,623.75","$62,355.37","$63,778.76","$41,401,116,964","$1,254,457,600,698"


In [ ]:
history_df.shape

(59, 7)

In [ ]:
history_df.to_csv('../data/raw/bitcoin_historical_data.csv', index=False)

In [ ]:
for currency in df['slug'][:2]:
    print(currency)

bitcoin
ethereum
